In [1]:
from dotenv import load_dotenv
load_dotenv()

import os, json, hashlib
import fitz
from groq import Groq
from neo4j import GraphDatabase

from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict, Any

from sentence_transformers import SentenceTransformer
import numpy as np

In [2]:
# ===== HF EMBEDDING VIA API (NO MODEL DOWNLOAD) =====
import requests

HF_TOKEN = os.environ.get("HF_TOKEN")
HF_EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # Reliable model
HF_API_URL = (
    f"https://router.huggingface.co/hf-inference/models/{HF_EMBED_MODEL}"
)
# Groq
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])

# Neo4j
neo4j_driver = GraphDatabase.driver(
    os.environ["NEO4J_URI"],
    auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
)

def get_embeddings_api(text):
    """Get embeddings using HuggingFace Inference API"""
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    payload = {"inputs": text, "options": {"wait_for_model": True}}
    
    try:
        response = requests.post(HF_API_URL, headers=headers, json=payload, timeout=2)
        
        if response.status_code == 200:
            result = response.json()
            # Handle different response formats
            if isinstance(result, list):
                if isinstance(result[0], list):
                    return result[0]  # Nested list
                return result
            return result
        else:
            # print(f"❌ API Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"❌ Exception: {str(e)}")
        return None

print("✅ Clients initialized (API mode - no model download)")

✅ Clients initialized (API mode - no model download)


In [3]:
def pdf_hash(path):
    with open(path, "rb") as f:
        return hashlib.sha256(f.read()).hexdigest()

def extract_text(pdf):
    doc = fitz.open(pdf)
    return "".join(p.get_text() for p in doc)

def safe_str(v):
    return v.strip() if isinstance(v, str) else None

def normalize_list(items, key=None):
    out = []
    for i in items:
        if isinstance(i, dict):
            val = i.get(key) if key else None
            if val:
                out.append(val)
        elif isinstance(i, str):
            out.append(i)
    return out

In [4]:
def print_contract_summary(data):
    """Enhanced summary printer with all details"""
    print("\n" + "="*80)
    print("📄 CONTRACT SUMMARY")
    print("="*80)

    print(f"\n📌 BASIC INFORMATION")
    print("-"*80)
    print(f"Title          : {data.get('title', 'N/A')}")
    print(f"File Name      : {data.get('file_name', 'N/A')}")
    print(f"Contract ID    : {data.get('contract_id', 'N/A')}")
    print(f"Governing Law  : {data.get('governing_law', 'N/A')}")

    # Parties
    print(f"\n👥 PARTIES ({len(data.get('parties', []))})")
    print("-"*80)
    for i, p in enumerate(data.get('parties', []), 1):
        print(f"  [{i}] {p}")

    # Important Dates
    print(f"\n📅 IMPORTANT DATES ({len(data.get('dates', []))})")
    print("-"*80)
    for i, d in enumerate(data.get('dates', []), 1):
        print(f"  [{i}] {d}")

    # Clauses with Risk Analysis
    print(f"\n⚖️ CLAUSE RISK ANALYSIS ({len(data.get('clauses', []))})")
    print("="*80)
    for i, c in enumerate(data.get("clauses", []), 1):
        print(f"\n[Clause {i}] {c.get('clause_name', 'Unnamed')}")
        print("-"*80)
        print(f"Summary      : {c.get('summary', 'N/A')}")
        print(f"\n🚨 Risk Level : {c.get('risk_level', 'N/A')}")
        print(f"Risk Reason  : {c.get('risk_reason', 'N/A')}")
        print(f"\n📋 Obligation : {c.get('obligation', 'N/A')}")
        print(f"💼 Liability  : {c.get('liability', 'N/A')}")
        print(f"\n🤖 AI Summary : {c.get('ai_summary', 'N/A')}")
        print("-"*80)
    
    print("\n" + "="*80)

In [5]:
class ContractState(TypedDict):
    pdf_path: str
    cid: str
    text: str
    embeddings: List[float]
    analysis: Dict[str, Any]

In [6]:
def pdf_extraction_agent(state: ContractState):
    print(f"\n📄 Extracting PDF: {state['pdf_path']}")
    text = extract_text(state["pdf_path"])
    return {
        **state,
        "text": text
    }

In [7]:
def embedding_agent(state: ContractState):
    print("🔢 Generating embeddings via HuggingFace API")

    # Limit chunk size for HF inference safety
    text_chunk = state["text"][:8000]

    emb = get_embeddings_api(text_chunk)

    # Fallback if API fails or returns bad data
    if (
        emb is None
        or not isinstance(emb, list)
        or len(emb) == 0
    ):
        print("⚠️ Using fallback embeddings")
        emb = [0.0] * 384  # all-MiniLM-L6-v2 → 384 dims

    # Handle nested response ([[...]])
    if isinstance(emb[0], list):
        emb = emb[0]

    # Final dimension guard (VERY important for Neo4j vector index)
    if len(emb) != 384:
        print(f"⚠️ Invalid embedding size {len(emb)}, forcing fallback")
        emb = [0.0] * 384

    print(f"   Embedding dimension: {len(emb)}")

    return {
        **state,
        "embeddings": emb
    }
def get_embeddings_api(text):
    headers = {
        "Authorization": f"Bearer {HF_TOKEN}",
        "Content-Type": "application/json"
    }

    payload = {
        "inputs": text,
        "options": {
            "wait_for_model": True
        }
    }

    try:
        response = requests.post(
            HF_API_URL,
            headers=headers,
            json=payload,
            timeout=30
        )

        if response.status_code == 200:
            result = response.json()

            # HF returns: [ [token_embeddings...] ]
            # We must MEAN POOL
            if isinstance(result, list) and isinstance(result[0], list):
                import numpy as np
                return np.mean(result, axis=0).tolist()

            print("⚠️ Unexpected HF response:", result)
            return None

        else:
            # print(f"❌ API Error: {response.status_code} - {response.text}")
            return None

    except Exception as e:
        print(f"❌ Exception: {e}")
        return None
    


In [8]:
# def get_embeddings_api(text):
#     headers = {
#         "Authorization": f"Bearer {HF_TOKEN}",
#         "Content-Type": "application/json"
#     }

#     payload = {
#         "inputs": text,
#         "options": {"wait_for_model": True}
#     }

#     try:
#         response = requests.post(
#             HF_API_URL,
#             headers=headers,
#             json=payload,
#             timeout=30
#         )

#         if response.status_code == 200:
#             result = response.json()

#             # token-level embeddings → mean pooling
#             if isinstance(result, list) and isinstance(result[0], list):
#                 import numpy as np
#                 return np.mean(result, axis=0).tolist()

#             print("⚠️ Unexpected HF response:", result)
#             return None

#         print(f"❌ HF API Error {response.status_code}: {response.text}")
#         return None

#     except Exception as e:
#         print(f"❌ HF Exception: {e}")
#         return None


In [9]:
def analysis_agent(state: ContractState):
    print("🧠 Analyzing contract via Groq LLM")

    # Enhanced prompt for better extraction
    prompt = f"""
You are a legal contract analyzer. Analyze the following contract and extract detailed information.

CRITICAL: Return ONLY valid JSON. No markdown, no explanations, just the JSON object.

{{
  "title": "Contract title",
  "parties": [
    {{"name": "Party 1 name", "role": "Role (e.g., Service Provider, Client)"}},
    {{"name": "Party 2 name", "role": "Role"}}
  ],
  "dates": [
    {{"type": "Effective Date", "value": "YYYY-MM-DD or as mentioned"}},
    {{"type": "Expiration Date", "value": "YYYY-MM-DD or as mentioned"}}
  ],
  "governing_law": "Jurisdiction and governing law",
  "clauses": [
    {{
      "clause_name": "Name of the clause",
      "summary": "Brief summary of what this clause says",
      "risk_level": "Low/Medium/High",
      "risk_reason": "Detailed explanation of why this risk level was assigned. Mention specific concerns, potential liabilities, or unfavorable terms.",
      "obligation": "Specific obligations this clause imposes on parties. Be detailed.",
      "liability": "What liabilities or penalties are mentioned in this clause. Include financial limits if any.",
      "ai_summary": "A comprehensive AI analysis of this clause including: 1) What it means in plain language, 2) Key takeaways, 3) Red flags or concerns, 4) Recommendations"
    }}
  ]
}}

IMPORTANT INSTRUCTIONS:
1. Extract ALL major clauses from the contract (aim for 5-10 clauses)
2. For risk_reason: Explain WHY you assigned that risk level with specific concerns
3. For ai_summary: Provide detailed analysis (at least 2-3 sentences)
4. Be thorough - don't leave fields empty
5. Focus on: payment terms, termination, liability, intellectual property, confidentiality, warranties, indemnification
6. ESCAPE all quotes inside strings properly using backslash
7. Do NOT include any text before or after the JSON object

CONTRACT TEXT:
{state["text"][:10000]}
"""

    try:
        res = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": "You are a JSON-only API. Return only valid JSON, no markdown or explanations."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=8000
        )

        content = res.choices[0].message.content.strip()
        
        # Clean the response
        # Remove markdown code blocks
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
        elif "```" in content:
            content = content.split("```")[1].split("```")[0].strip()
        
        # Remove any leading/trailing whitespace
        content = content.strip()
        
        # Try to find JSON object boundaries
        if not content.startswith("{"):
            start = content.find("{")
            if start != -1:
                content = content[start:]
        
        if not content.endswith("}"):
            end = content.rfind("}")
            if end != -1:
                content = content[:end+1]
        
        # Attempt to parse
        try:
            analysis = json.loads(content)
        except json.JSONDecodeError as e:
            print(f"⚠️ JSON parsing failed at position {e.pos}: {e.msg}")
            print(f"Problematic content around error: ...{content[max(0,e.pos-50):e.pos+50]}...")
            
            # Try to fix common issues
            import re
            
            # Fix unescaped quotes in strings
            content = re.sub(r'(?<!\\)"(?=\w)', r'\\"', content)
            
            # Try parsing again
            try:
                analysis = json.loads(content)
                print("✅ JSON fixed and parsed successfully")
            except:
                print("❌ Could not fix JSON, using fallback structure")
                # Fallback structure
                analysis = {
                    "title": "Contract Analysis",
                    "parties": [{"name": "Party A", "role": "Unknown"}, {"name": "Party B", "role": "Unknown"}],
                    "dates": [{"type": "Effective Date", "value": "Not specified"}],
                    "governing_law": "Not specified",
                    "clauses": [
                        {
                            "clause_name": "General Terms",
                            "summary": "Contract terms extracted from document",
                            "risk_level": "Medium",
                            "risk_reason": "Unable to fully analyze due to parsing error. Manual review recommended.",
                            "obligation": "Review document manually for obligations",
                            "liability": "Review document manually for liabilities",
                            "ai_summary": "Automated analysis encountered an error. This contract requires manual legal review to identify all terms, conditions, and potential risks."
                        }
                    ]
                }
        
        print(f"✅ Analysis complete - Found {len(analysis.get('clauses', []))} clauses")

        return {
            **state,
            "analysis": analysis
        }
        
    except Exception as e:
        print(f"❌ Analysis error: {str(e)}")
        # Return minimal fallback
        return {
            **state,
            "analysis": {
                "title": "Error in Analysis",
                "parties": [],
                "dates": [],
                "governing_law": "Unknown",
                "clauses": []
            }
        }

In [10]:
def store_graph_agent(state: ContractState):
    data = state["analysis"]
    cid = state["cid"]
    filename = os.path.basename(state["pdf_path"])
    embeddings = state["embeddings"]

    print("🗄️ Storing into Neo4j with vector embeddings")

    with neo4j_driver.session() as s:

        # Contract with embeddings
        s.run("""
        MERGE (c:Contract {id:$id})
        SET c.title=$title,
            c.file_name=$file,
            c.governing_law=$law,
            c.embedding=$emb
        """, id=cid, title=data.get("title", "Unknown Contract"), file=filename, 
             law = data.get("governing_law", "Not Specified"), emb=embeddings)

        # Parties with roles
        for p in data.get("parties", []):
            if isinstance(p, dict):
                s.run("""
                MERGE (o:Organization {name:$name})
                SET o.role=$role
                WITH o
                MATCH (c:Contract {id:$id})
                MERGE (o)-[:IS_PARTY_TO]->(c)
                """, name=p.get("name"), role=p.get("role"), id=cid)

        # Dates with types
        for d in data.get("dates", []):
            if isinstance(d, dict):
                s.run("""
                MERGE (dt:ImportantDate {value:$v})
                SET dt.type=$type
                WITH dt
                MATCH (c:Contract {id:$id})
                MERGE (c)-[:HAS_DATE]->(dt)
                """, v=d.get("value"), type=d.get("type"), id=cid)

        # Clauses with all details
        for cl in data.get("clauses", []):
            # Generate embedding for each clause via API
            clause_text = f"{cl.get('clause_name', '')} {cl.get('summary', '')}"
            clause_emb = get_embeddings_api(clause_text)
            
            if clause_emb is None:
                clause_emb = [0.0] * 768
            
            # Normalize if nested
            if isinstance(clause_emb[0], list):
                clause_emb = clause_emb[0]
            
            s.run("""
            MATCH (c:Contract {id:$id})
            CREATE (cl:Clause {
                name:$n, 
                summary:$s,
                embedding:$emb
            })

            MERGE (r:Risk {level:$rl})
            MERGE (rr:RiskReason {text:$rr})
            MERGE (o:Obligation {text:$ob})
            MERGE (l:Liability {text:$li})
            MERGE (ai:AISummary {text:$ai})

            CREATE (c)-[:HAS_CLAUSE]->(cl)
            CREATE (cl)-[:HAS_RISK]->(r)
            CREATE (cl)-[:HAS_REASON]->(rr)
            CREATE (cl)-[:HAS_OBLIGATION]->(o)
            CREATE (cl)-[:HAS_LIABILITY]->(l)
            CREATE (cl)-[:HAS_AI_SUMMARY]->(ai)
            """,
            id=cid,
            n=safe_str(cl.get("clause_name")),
            s=safe_str(cl.get("summary")),
            rl=safe_str(cl.get("risk_level")),
            rr=safe_str(cl.get("risk_reason")),
            ob=safe_str(cl.get("obligation")),
            li=safe_str(cl.get("liability")),
            ai=safe_str(cl.get("ai_summary")),
            emb=clause_emb
            )

    print("✅ Stored successfully with vector embeddings")
    return state

In [11]:
graph = StateGraph(ContractState)

graph.add_node("extract", pdf_extraction_agent)
graph.add_node("embed", embedding_agent)
graph.add_node("analyze", analysis_agent)
graph.add_node("store", store_graph_agent)

graph.set_entry_point("extract")

graph.add_edge("extract", "embed")
graph.add_edge("embed", "analyze")
graph.add_edge("analyze", "store")
graph.add_edge("store", END)

workflow = graph.compile()
print("✅ LangGraph workflow ready")

✅ LangGraph workflow ready


In [12]:
def retrieve_contract_from_db(contract_id):
    """
    Retrieve complete contract details from Neo4j database
    """
    print(f"\n🔍 Retrieving contract: {contract_id}")
    
    with neo4j_driver.session() as s:
        # Get contract with all related data
        result = s.run("""
        MATCH (c:Contract {id:$id})
        OPTIONAL MATCH (c)<-[:IS_PARTY_TO]-(org:Organization)
        OPTIONAL MATCH (c)-[:HAS_DATE]->(dt:ImportantDate)
        OPTIONAL MATCH (c)-[:HAS_CLAUSE]->(cl:Clause)
        OPTIONAL MATCH (cl)-[:HAS_RISK]->(r:Risk)
        OPTIONAL MATCH (cl)-[:HAS_REASON]->(rr:RiskReason)
        OPTIONAL MATCH (cl)-[:HAS_OBLIGATION]->(o:Obligation)
        OPTIONAL MATCH (cl)-[:HAS_LIABILITY]->(l:Liability)
        OPTIONAL MATCH (cl)-[:HAS_AI_SUMMARY]->(ai:AISummary)
        
        RETURN c.title as title,
               c.file_name as file_name,
               c.id as contract_id,
               c.governing_law as governing_law,
               collect(DISTINCT org.name) as parties,
               collect(DISTINCT dt.value) as dates,
               collect(DISTINCT {
                   clause_name: cl.name,
                   summary: cl.summary,
                   risk_level: r.level,
                   risk_reason: rr.text,
                   obligation: o.text,
                   liability: l.text,
                   ai_summary: ai.text
               }) as clauses
        """, id=contract_id)
        
        record = result.single()
        if record:
            data = {
                "title": record["title"],
                "file_name": record["file_name"],
                "contract_id": record["contract_id"],
                "governing_law": record["governing_law"],
                "parties": [p for p in record["parties"] if p],
                "dates": [d for d in record["dates"] if d],
                "clauses": [c for c in record["clauses"] if c.get("clause_name")]
            }
            print("✅ Contract retrieved successfully")
            return data
        else:
            print("❌ Contract not found")
            return None

def retrieve_all_contracts():
    """
    Retrieve all contracts from database
    """
    print("\n📚 Retrieving all contracts...")
    
    with neo4j_driver.session() as s:
        result = s.run("""
        MATCH (c:Contract)
        RETURN c.id as id, c.title as title, c.file_name as file_name
        """)
        
        contracts = []
        for record in result:
            contracts.append({
                "id": record["id"],
                "title": record["title"],
                "file_name": record["file_name"]
            })
        
        print(f"✅ Found {len(contracts)} contracts")
        return contracts

def search_similar_clauses(query_text, top_k=5):
    """
    Search for similar clauses using vector embeddings
    """
    print(f"\n🔍 Searching for clauses similar to: '{query_text}'")
    
    # Generate embedding for query via API
    query_emb = get_embeddings_api(query_text)
    
    if query_emb is None:
        print("❌ Could not generate query embedding")
        return []
    
    # Normalize if nested
    if isinstance(query_emb[0], list):
        query_emb = query_emb[0]
    
    with neo4j_driver.session() as s:
        # Get all clauses with embeddings
        result = s.run("""
        MATCH (c:Contract)-[:HAS_CLAUSE]->(cl:Clause)
        WHERE cl.embedding IS NOT NULL
        RETURN c.title as contract_title,
               cl.name as clause_name,
               cl.summary as summary,
               cl.embedding as embedding
        """)
        
        clauses = []
        for record in result:
            # Calculate cosine similarity
            emb = np.array(record["embedding"])
            query = np.array(query_emb)
            
            # Handle different embedding dimensions
            if len(emb) != len(query):
                continue
            
            similarity = np.dot(emb, query) / (np.linalg.norm(emb) * np.linalg.norm(query))
            
            clauses.append({
                "contract": record["contract_title"],
                "clause": record["clause_name"],
                "summary": record["summary"],
                "similarity": float(similarity)
            })
        
        # Sort by similarity
        clauses.sort(key=lambda x: x["similarity"], reverse=True)
        
        print(f"\n📊 Top {top_k} similar clauses:")
        for i, clause in enumerate(clauses[:top_k], 1):
            print(f"\n[{i}] Similarity: {clause['similarity']:.4f}")
            print(f"    Contract: {clause['contract']}")
            print(f"    Clause: {clause['clause']}")
            print(f"    Summary: {clause['summary']}")
        
        return clauses[:top_k]

In [ ]:
# ===== PROCESS CONTRACTS =====
pdfs = [
    "Legal-Services-Agreement.pdf",
    "Employment_contract.pdf",
    "sample_contract.pdf"
]

contract_ids = []

for pdf in pdfs:
    cid = pdf_hash(pdf)
    contract_ids.append(cid)

    print("\n" + "="*80)
    print(f"🚀 Processing: {pdf}")
    print("="*80)

    workflow.invoke({
        "pdf_path": pdf,
        "cid": cid,
        "text": "",
        "embeddings": [],
        "analysis": {},
    })


🚀 Processing: Legal-Services-Agreement.pdf

📄 Extracting PDF: Legal-Services-Agreement.pdf
🔢 Generating embeddings via HuggingFace API
⚠️ Using fallback embeddings
   Embedding dimension: 384
🧠 Analyzing contract via Groq LLM
✅ Analysis complete - Found 20 clauses
🗄️ Storing into Neo4j with vector embeddings
✅ Stored successfully with vector embeddings


In [14]:
# ===== RETRIEVE AND DISPLAY RESULTS =====

print("\n\n" + "#"*80)
print("# RETRIEVING STORED CONTRACTS FROM DATABASE")
print("#"*80)

# Show all contracts
all_contracts = retrieve_all_contracts()

# Retrieve and display each processed contract
for cid in contract_ids:
    contract_data = retrieve_contract_from_db(cid)
    if contract_data:
        print_contract_summary(contract_data)



################################################################################
# RETRIEVING STORED CONTRACTS FROM DATABASE
################################################################################

📚 Retrieving all contracts...
✅ Found 5 contracts

🔍 Retrieving contract: 1de79b4ffc94b68989ab72c79d929fc5169e8c48b1ea78fc7ee7c8bcd0b1c7d2
✅ Contract retrieved successfully

📄 CONTRACT SUMMARY

📌 BASIC INFORMATION
--------------------------------------------------------------------------------
Title          : Legal Services Agreement
File Name      : Legal-Services-Agreement.pdf
Contract ID    : 1de79b4ffc94b68989ab72c79d929fc5169e8c48b1ea78fc7ee7c8bcd0b1c7d2
Governing Law  : California law

👥 PARTIES (2)
--------------------------------------------------------------------------------
  [1] Law Firm
  [2] Client

📅 IMPORTANT DATES (1)
--------------------------------------------------------------------------------
  [1] Not specified

⚖️ CLAUSE RISK ANALYSIS (20)

[Clause 1] Iden

In [ ]:
# ===== VECTOR SIMILARITY SEARCH EXAMPLE =====

print("\n\n" + "#"*80)
print("# VECTOR SIMILARITY SEARCH")
print("#"*80)

# Example searches
search_queries = [
    "payment terms and conditions",
    "liability and indemnification",
    "termination clause"
]

for query in search_queries:
    search_similar_clauses(query, top_k=3)
    print("\n" + "-"*80)

In [ ]:
# ===== CLEANUP =====
import atexit
atexit.register(lambda: neo4j_driver.close())
print("🔒 Neo4j connection will close on exit")
